# Анализ базы данных книг для мобильного приложения.

***Задача: проанализировать базу данных***

***Цель: сформулировать ценностное предложение для нового продукта.***

**Описание данных**

Таблица `books`:
- `book_id` - идентификатор книги
- `author_id` - идентификатор пользователя
- `title` - название книги
- `num_pages` - количество страниц
- `publication_date` - дата публикации книги
- `publisher_id` - идентификатор издателя

Таблица `authors`:
- `author_id` - идентификатор автора
- `author` - имя автора

Таблица `publishers`:
- `publisher_id` - идентификатор издательства
- `publisher` - название издательства

Таблица `ratings`:

- `rating_id` - идентификатор оценки
- `book_id` - идентификатор книги
- `username` - имя пользователя,оставившего оценку
- `rating` - оценка книги

Таблица `reviews`:
- `review_id` - идентификатор обзора
- `book_id` - идентификатор книги
- `username` - имя автора обзора
- `text` - текст обзора

План работы:
1. Подключить библиотеки, написать код для обработки SQL-запросов
2. Исследовать таблицы, вывести первые строки
3. Решить задачи с помощью SQL-кода:
   - Посчитать, сколько книг вышло после 1 января 2000 года
   - Для каждой книги посчитатье количество обзоров и среднюю оценку
   - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры
   - Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками
   - Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок
4. Сделать выводы по задачам.

##  Подключение библиотек, написание кода для запросов.

### Подкючение библиотек.

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

### Код для подключения к базе данных и функция для запросов.

In [2]:
# устанавливаем параметры
db_config = {
            'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db' # название базы данных
            }
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

##  Исследование таблиц.

Поскольку требуется произвести одни и те же операции с несколькими таблицами, напишем функции для упрощения кода.

In [4]:
def summary(table):
    '''
    получение первых 5 строк из таблицы
    '''
    query = f'SELECT * FROM {table} LIMIT 5'
    return get_sql_data(query)

In [5]:
def dist_count(table, field):
    '''
    получение количества уникальных значений в поле
    '''
    query = f'SELECT COUNT(DISTINCT {field}) FROM {table}'
    return get_sql_data(query)

In [6]:
def min_max(table, field):
    '''
    получение минимального и максимального значений в поле
    '''
    query = f'SELECT MIN ({field}), MAX ({field})  FROM {table}'
    return get_sql_data(query)

In [7]:
def avg_func(table, field):
    '''
    получение среднего значения в поле
    '''
    query = f'SELECT ROUND(AVG({field}), 2) FROM {table}'
    return get_sql_data(query)

In [8]:
def avg_count(table, field1, field2):
    '''
    получение количества уникальных значений в поле
    '''
    query = f'SELECT ROUND(COUNT(DISTINCT {field1})::numeric / COUNT(DISTINCT {field2}), 2) AS avg_cnt FROM {table}'
    return get_sql_data(query)

###  Таблица Books.

**Первые 5 строк таблицы.**

In [9]:
summary('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Количество строк в таблице(количество книг).**

Поскольку каждое значение значение book_id само по себе уникально, для определения количества строк подойдет функция, которая содержит DISTINCT

In [10]:
dist_count('books', 'book_id')

,count
0,1000


**Кличество уникальных названий книг**

In [11]:
dist_count('books', 'title')

,count
0,999


Количество уникальных названий на 1 меньше, чем книг всего. Значит какая-то книга была издана дважды. Сгруппируем по названию, найдем эту книгу.

In [12]:
query = '''SELECT title, COUNT(title) FROM books GROUP BY title
ORDER BY count DESC LIMIT 1
'''
get_sql_data(query)

,title,count
0,Memoirs of a Geisha,2


In [13]:
query = '''SELECT * FROM books
WHERE title = 'Memoirs of a Geisha'
'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Мемуары гейши были выпущены разными издательствами в разном формате примерно в одно время. Теория подтвердилась.

**Количество уникальных авторов.**

In [14]:
dist_count('books', 'author_id')

,count
0,636


**Количество уникальных издательств.**

In [15]:
dist_count('books', 'publisher_id')

,count
0,340


**Количество страниц.**

In [16]:
min_max('books', 'num_pages')

,min,max
0,14,2690


In [17]:
avg_func('books', 'num_pages')

,round
0,389.11


**Дата публикации**

In [18]:
min_max('books', 'publication_date')

,min,max
0,1952-12-01,2020-03-31


**Выводы по таблице `books`**
Таблица содержит:
- 6 столбцов,
- 1000 строк,
- 999 уникальных названий книг,
- 636 авторов книг,
- 340 издательств,

Количество страниц в книгах находится в диапазоне от 14 до 2690 страниц, в среднем 389 страниц.

Периоды издания книг с 1 декабря 1952 года по 31 марта 2020.

###  Таблица Authors.

**Первые 5 строк таблицы.**

In [19]:
summary('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Количество строк в таблице.**

In [20]:
dist_count('authors', 'author_id')

,count
0,636


Проверим, что количество строк совпадает с количеством уникальных имен авторов для понимания, нет ли среди авторов полных тёзок.

In [21]:
dist_count('authors', 'author')

,count
0,636


**Выводы по таблице `authors`:**
- количество строк в таблице равно 636,
- полных тёзок нет,
- количество авторов совпадает с количеством авторов в таблице `books`.

### Таблица Publishers.

**Первые 5 строк таблицы.**

In [22]:
summary('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Количество строк в таблице.**

In [23]:
dist_count('publishers', 'publisher_id')

,count
0,340


In [24]:
dist_count('publishers', 'publisher')

,count
0,340


**Выводы по таблице `publishers`:**
- количество строк в таблице равно 340,
- совпадающих по названию издательств нет,
- количество издательств совпадает с количеством издательств в таблице `books`.

### Таблица Ratings.

**Первые 5 строк таблицы.**

In [25]:
summary('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Количество строк в таблице.**

In [26]:
dist_count('ratings', 'rating_id')

,count
0,6456


**Количество книг с рейтингом.**

In [27]:
dist_count('ratings', 'book_id')

,count
0,1000


**Количество уникальных пользователей, оставивших оценку.**

In [28]:
dist_count('ratings', 'username')

,count
0,160


**Максимальный, минимальный и средний рейтинг книг.**

In [29]:
min_max('ratings', 'rating')

,min,max
0,1,5


In [30]:
avg_func('ratings', 'rating')

,round
0,3.93


**Среднее количество оценок на пользователя.**

In [31]:
avg_count('ratings', 'rating_id', 'username')

,avg_cnt
0,40.35


**Выводы по таблице `Ratings.`**

Таблица содержит:
- 6456 строк,
- 1000 книг с рейтингом(все книги имеют рейтинг),
- 160 пользователей, оставивших оценку,
- каждый пользователь в среднем поставил 40,35 оценок.

Минимальный рейтинг - 1, максимальный - 5, средний рейтинг - 3,93

### Таблица Reviews.

**Первые 5 строк таблицы.**

In [32]:
summary('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Количество строк в таблице.**

In [33]:
dist_count('reviews', 'review_id')

,count
0,2793


**Количество книг с обзорами.**

In [34]:
dist_count('reviews', 'book_id')

,count
0,994


**Количество уникальных пользователей, написавших обзор.**

In [35]:
dist_count('reviews', 'username')

,count
0,160


In [36]:
avg_count('reviews', 'review_id', 'username')

,avg_cnt
0,17.46


**Выводы по таблице `Reviews`**

Таблица содержит:
- 2793 строки,
- 994 книги имеют отзывы(6 книг остались без отзывов)
- 160 пользователей,написавших отзыв
- каждый пользователь в среднем оставил 17,46 отзывов.

## Решение задач с посщью SQL-кода

### Количество книг, выпущенных после 1 января 2000 года

In [37]:
query = '''
SELECT COUNT(book_id) 
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


**Вывод:**  после 1 января 2000 года было издано 819 книг.

###  Количество обзоров и средний рейтинг каждой книги.

Для решения данной задачи использованы 2 табличных выражения и последующее объединение. Поскольку не все книги имеют отзывы, образовались пустые строки, их меняем на 0 в сводной таблице.

In [38]:
#объединяем данные в общую таблицу
query = '''
WITH rate AS (
    SELECT rat.book_id, AVG(rating) AS avg_rating
    FROM ratings AS rat
    GROUP BY book_id
),
reviews AS (
    SELECT rev.book_id, COUNT(*) AS reviews_count
    FROM reviews AS rev
    GROUP BY book_id
)
SELECT b.title,
       COALESCE(re.reviews_count, 0) AS reviews_count,
       ROUND(ra.avg_rating, 2) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS re ON re.book_id = b.book_id
LEFT JOIN rate AS ra ON ra.book_id = b.book_id
ORDER BY avg_rating DESC, reviews_count DESC;
'''
get_sql_data(query)

,title,reviews_count,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,Moneyball: The Art of Winning an Unfair Game,3,5.00
2,School's Out—Forever (Maximum Ride #2),3,5.00
3,The Woman in Black,2,5.00
4,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


**Вывод:** Задача решена. В итоговой таблице 1000 строк, что соответствует количесву книг. Для каждой книги добавлено количество отзывов и средний рейтинг. Книги отсортированы по уменьшению количества отзывов и среднему рейтингу. Первую строчку в рейтинге занимает A Dirty Job (Grim Reaper #1)	 с 4 обзорами и средним рейтингом 5.

###  Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [39]:
query = '''
SELECT p.publisher, COUNT(b.book_id) AS book_count
FROM publishers AS p
RIGHT JOIN books AS b ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 1
'''
get_sql_data(query)

,publisher,book_count
0,Penguin Books,42


**Вывод:** самое активное издательство - Penguin Books, которое выпустило 42 книги.

### Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [40]:
query = '''
SELECT a.author, ROUND(AVG(r.rating), 2) AS avg_rating
FROM books AS b
JOIN authors AS a ON a.author_id=b.author_id
JOIN ratings AS r ON r.book_id=b.book_id
WHERE b.book_id IN(SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(*) >= 50)
GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT 5
'''
get_sql_data(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


**Вывод:** Лидером по средней оценке среди тех, чьи книги получили более 50 оценок является J.K. Rowling/Mary GrandPré со средним рейтингом 4,29

###  Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [41]:
query = '''
SELECT AVG(review_count)
FROM (SELECT COUNT(review_id) AS review_count
        FROM reviews
        WHERE username IN (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48)
    GROUP BY username) AS review_count;
'''
get_sql_data(query)

,avg
0,24.0


## Итоговые выводы.

По результатам изучения 5 имеющихся таблиц получили следующие результаты:
- в наличии имеется 1000 книг,
- диапазон написания книг от 1 декабря 1952до 31 марта 2020 года,
- среднее количество страниц в книге -389,
- эти книги написанали 636 авторов,
- книги издавали 340 издательств,
- книги оценивали и писали отзывы 160 читателей,
- все книги имеют оценку,
- на 6 книг не написан отзыв,
- каждый читатель в среднем оставляет 40,35 оценок
- каждый читатель в среднем пишет 17,46 отзывов

Выводы по поставленным задачам:
- После 1 января 2000 года было издано 819 книг,
- Посчитаны общее количество обзоров и средняя оценка каждой книги. Первую строчку в рейтинге занимает A Dirty Job (Grim Reaper #1) с 4 обзорами и средним рейтингом 5,
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books и 42 выпущенные книги,
- Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - J.K. Rowling/Mary GrandPré со средним рейтингом 4,29,
- В среднем пользователи, которые поставили больше 48 оценок пишут 24 отзыва.